# GLENS - LOGISTIC REGRESSION

In [ ]:
import numpy as np
import pandas as pd
import random
import xarray as xr
import scipy.stats as stats
import random

import time
import os.path
from os import path
import subprocess
import copy as copy
import pickle
import tensorflow as tf

import seaborn as sns
import palettable

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import cartopy as ct
import cartopy.crs as ccrs
import cmocean as cmocean
from mpl_toolkits.axes_grid1 import make_axes_locatable
import palettable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import gaussian_filter

import experiment_settings, plots

import imp

# Plotting
mpl.rcParams['figure.dpi']= 150
dpiFig = 300.
CL = 0.
mapProj = ct.crs.EqualEarth(central_longitude = CL)

# random numbers
%env PYTHONHASHSEED=99
np.random.seed(99)
tf.random.set_seed(99)

In [ ]:
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")  
print(f"tensorflow version = {tf.__version__}")  

In [ ]:
EXP_NAME = 'exp2_t'#'exp0_tx90p' #'exp1_r95ptot' 'exp2_t'
ENS_SEED = 3529
NET_SEED = 2222

#-------------------------------------------------------
settings = experiment_settings.get_settings(EXP_NAME)
display(settings)

fileDir = 'data/GLENS/'

In [ ]:
if(settings["var"]=='TX90p' or settings["var"]=='TX10p' or settings["var"]=='R95pTOT'):
    extremeBool = True    
else:
    extremeBool = False

## Get data loaded and ready

In [ ]:
df_mask = xr.open_dataset('data/GLENS/landSeaMask.r90x45.nc')
# print(df_mask)
landSeaMask = df_mask['TN10p'].values[0,0,:,:]
lats = df_mask['lat']
lons = df_mask['lon']
df_mask.close()

landSeaMask[np.isnan(landSeaMask)==False] = 1.
landSeaMask[np.isnan(landSeaMask)==True] = np.nan
landSeaMask = landSeaMask.reshape(len(lats)*len(lons),)
landSeaMask

In [ ]:
if(extremeBool == False):
    middletext = '.r90x45.shift.annual'

#     print('** control has different months for the same year **')    
    runType = 'control'
    dsc = xr.open_mfdataset(fileDir + runType + '*.' + settings["var"] + middletext + '.nc', concat_dim='ensmember', combine='nested')    
    time_dsc = dsc['time.year']
    
    runType = 'feedback'
    dsf = xr.open_mfdataset(fileDir + runType + '*.' + settings["var"] + middletext + '.nc', concat_dim='ensmember', combine='nested')
    time_dsf = dsf['time.year']
    
    if(settings["var"]=='T'):
        dsf = dsf.sel(lev=1000.)
        dsc = dsc.sel(lev=1000.)    
    
else:    # get the extremes data
    varFile = 'TREFHTMX'
    if(settings["var"]=='R95pTOT'):
        varFile = 'PRECT'

    middletext = '.swap.r90x45'
    runType = 'control'
    filename = fileDir + 'b.e15.B5505C5WCCML45BGCR.f09_g16.' + runType + '.extreme.cam.h3.' + varFile + '.20100101-20971231' + middletext + '.nc'
    dsc = xr.open_dataset(filename)
    dsc = dsc.transpose('ensemble', 'time', 'lat', 'lon')
    time_dsc = np.arange(2010, 2098, 1)
    
    runType = 'feedback'
    filename = fileDir + 'b.e15.B5505C5WCCML45BGCR.f09_g16.' + runType + '.extreme.cam.h3.' + varFile + '.20200101-20991231' + middletext + '.nc'
    dsf = xr.open_dataset(filename)
    dsf = dsf.transpose('ensemble', 'time', 'lat', 'lon')
    time_dsf = np.arange(2020,2100,1)
#------------------------------------------------------------------------
# CONTROL
itime = np.where((time_dsc>=2021) & (time_dsc<=2097))[0]    
featuresC = dsc[settings["var"]][:,itime,:,:].values.astype('float')
time_dsc = time_dsc[itime]

if(featuresC.shape[0]==21):
    featuresC = featuresC[:-1,:,:,:]
elif(featuresC.shape[0]==3):
    filename = fileDir + 'b.e15.B5505C5WCCML45BGCR.f09_g16.' + 'control' + '.extreme.cam.h3.' + varFile + '.20210101-20301231' + middletext + '.nc'
    dsc20 = xr.open_dataset(filename)
    dsc20 = dsc20.transpose('ensemble', 'time', 'lat', 'lon')
    featuresC20 = dsc20[settings["var"]].values.astype('float')
    hold = np.nan*np.empty((20,77,45,90))
    hold[:3,:,:,:] = featuresC
    hold[3:,:featuresC20.shape[1],:,:] = featuresC20[3:,:,:,:]
    featuresC = copy.deepcopy(hold)
#------------------------------------------------------------------------    
# FEEDBACK
itime = np.where((time_dsf>=2021) & (time_dsf<=2097))[0]   
featuresF = dsf[settings["var"]][:,itime,:,:].values.astype('float')
time_dsf = time_dsf[itime]
if(featuresF.shape[0]==21):
    featuresF = featuresF[:-1,:,:,:]
    
featuresC_raw = copy.deepcopy(featuresC)
featuresF_raw = copy.deepcopy(featuresF)
print(featuresC_raw.shape)
print(featuresF_raw.shape)    

# Summarize predictions across models

In [ ]:
with open("saved_predictions/" + settings["exp_name"] + ".pickle", 'rb') as handle:
    exp_dict = pickle.load(handle)
exp_dict.keys()

In [ ]:
imp.reload(plots)

fig = plt.figure(figsize=(10,4.5))
ax1 = plt.subplot(2,1,1)
ax2 = plt.subplot(2,1,2)

plots.plot_dots_seeds(ax1,ax2,exp_dict,time_dsc,settings)
# #--------------------------------------------------------
plt.tight_layout()
plt.savefig('figures/' + settings["exp_name"] + '_seeds_dots_summary.png', dpi=dpiFig, bbox_inches = 'tight')
plt.show()


# Explore specific model

In [ ]:
filename = settings["exp_name"] + '_ensSeed' + str(ENS_SEED) + '_netSeed' + str(NET_SEED)
model = tf.keras.models.load_model('saved_models/model_' + filename + '.h5')

In [ ]:
if(settings["var"]=='R95pTOT'):
    bold_year = 2040
else:
    bold_year = 2030

bias = model.get_weights()[1]
weights = model.get_weights()[0]

y_pred_like_train = exp_dict[filename]["y_pred_like_train"]
y_pred_train = exp_dict[filename]["y_pred_train"]
acc_train = exp_dict[filename]["acc_train"]
y_pred_like_test = exp_dict[filename]["y_pred_like_test"]
y_pred_test = exp_dict[filename]["y_pred_test"]
acc_test = exp_dict[filename]["acc_test"]
valNum = 4


corr_C = np.nansum(acc_test[:valNum,:], axis=0)    # plot testing only
corr_F = np.nansum(acc_test[valNum:,:], axis=0)   # plot testing only        
count_C = np.count_nonzero(~np.isnan(acc_test[:valNum,:]), axis=0)    # plot testing only
count_F = np.count_nonzero(~np.isnan(acc_test[valNum:,:]), axis=0)   # plot testing only

print(count_C)
print(count_F)

count_combine = count_C+count_F
corr_combine = corr_C+corr_F

year_list = np.arange(2021,2098)
first = True
for iy,year in enumerate(year_list):
    acc = np.round(np.sum(corr_combine[iy:]/np.sum(count_combine[iy:])),3)
    if(acc>=0.97 and first==True):
        print('****')
        first=False
    print(year, year-2020, str(acc))

In [ ]:
rng = np.random.default_rng(ENS_SEED)    
valNum = settings["n_train_val_test"][1]
val_member = rng.choice([0,1,2],1, replace=False)
train_members = rng.choice(np.arange(3,np.shape(featuresC_raw)[0]),np.shape(featuresC_raw)[0]-3, replace=False)
train_members = np.append(np.setdiff1d([0,1,2],[val_member]),train_members)
number_list = np.append(train_members,val_member)   

#-------------------
print('ensemble list = ' + str(number_list) + '\n')
featuresC = featuresC_raw[number_list,:,:,:]
featuresF = featuresF_raw[number_list,:,:,:]

#-------------------
# train/test split
print('training members = ' + str(number_list[:-valNum]))
print('validate members = ' + str(number_list[-valNum:]))
featuresF_train = featuresF[:-valNum]
featuresC_train = featuresC[:-valNum]
featuresF_test = featuresF[-valNum:]
featuresC_test = featuresC[-valNum:]

labelsC_train = np.ones((featuresC_train.shape[0],featuresC_train.shape[1]))*0
labelsF_train = np.ones((featuresF_train.shape[0],featuresF_train.shape[1]))*1
labelsC_test = np.ones((featuresC_test.shape[0],featuresC_test.shape[1]))*0
labelsF_test = np.ones((featuresF_test.shape[0],featuresF_test.shape[1]))*1

#-------------------
# concatenate
features_train = np.append(featuresC_train,featuresF_train,axis=0)
features_test = np.append(featuresC_test,featuresF_test,axis=0)
labels_train = np.append(labelsC_train,labelsF_train,axis=0)
labels_test = np.append(labelsC_test,labelsF_test,axis=0)

#-------------------
# standardize
featuresMean = np.nanmean(features_train,axis=(0,1))
featuresStd = np.nanstd(features_train,axis=(0,1))

features_train = (features_train-featuresMean)/featuresStd
features_test = (features_test-featuresMean)/featuresStd

# convert labels to categorical
y_train = tf.keras.utils.to_categorical(labels_train)
y_test = tf.keras.utils.to_categorical(labels_test)

# grab 2021-2030 for training
itime_train = np.where(time_dsc<=2097)[0]
X_train = features_train[:,itime_train,:,:]
X_test = features_test[:,itime_train,:,:]
y_train = y_train[:,itime_train,:]
y_test = y_test[:,itime_train,:]

# flatten the training and testing 
X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1],X_train.shape[2]*X_train.shape[3])
X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2]*X_test.shape[3])
y_train = y_train.reshape((y_train.shape[0]*y_train.shape[1],y_train.shape[2]))
y_test = y_test.reshape((y_test.shape[0]*y_test.shape[1],y_test.shape[2]))

if(settings["land_only"]==True):
    X_train = X_train*landSeaMask
    X_test = X_test*landSeaMask
    landText = '_landOnly'
else:
    landText = ''
    X_train = X_train
    X_test = X_test

if(settings["remove_mean"]==True):
    print('removing the mean...')
    X_train = X_train - np.nanmean(X_train,axis=1)[:,np.newaxis]
    X_test = X_test - np.nanmean(X_test,axis=1)[:,np.newaxis]    

# replace NaN with zeros    
X_train[np.isnan(X_train)] = 0.
X_test[np.isnan(X_test)] = 0.

# make output predict whether from control
y_train = y_train[:,1:]
y_test = y_test[:,1:]
#=========================================================================


## Plot the panels

In [ ]:
imp.reload(plots)
from matplotlib.gridspec import GridSpec
titleSize=16
fig = plt.figure(figsize=(8*1,3*2.5))
gs = GridSpec(3, 4, height_ratios=[1.25,8,8])

# ACCURACY
ax = fig.add_subplot(gs[0,:])
plots.plot_dots(ax,corr_F,corr_C,count_F,count_C,time_dsc,marker='o',max_year=2080,bold_year=bold_year)

# # WEIGHTS
# ax = fig.add_subplot(gs[1,0:2],projection=mapProj)
# plots.plot_weights(ax,weights,landSeaMask,dsf)
# plt.title('(b) ' + settings["var"], fontsize=titleSize)   

# SNR
# ax = plt.subplot(3,2,4,projection=mapProj)
ax = fig.add_subplot(gs[1,1:3],projection=mapProj)
title_name = plots.plot_snr(ax,featuresC,featuresF,landSeaMask,time_dsc,dsf,settings)
plt.title('(b) ' + title_name, fontsize=16)            

# CONTRIBUTIONS
ax1 = fig.add_subplot(gs[2,0:2],projection=mapProj)
ax2 = fig.add_subplot(gs[2,2:4],projection=mapProj)
plots.plot_contributions(ax1,ax2,model,features_train,features_test,X_train,X_test,y_train,y_test,landSeaMask,time_dsc,dsf,settings)


# --- wrap up ----
# plt.tight_layout()
plt.savefig('figures/' + filename + '_map_panels.png', dpi=dpiFig, bbox_inches = 'tight')
plt.show()



In [ ]:
# plot signal-to-noise patterns
imp.reload(plots)
from matplotlib.gridspec import GridSpec
titleSize=16
fig = plt.figure(figsize=(8*1,2*2.5))
gs = GridSpec(2, 2)

if settings["var"]=="R95pTOT":
    maxVal = 0.25
else:    
    maxVal = .75

plot_year = 2021
ax1 = fig.add_subplot(gs[0,0],projection=mapProj)
plots.plot_noise(ax1,featuresC,featuresF,featuresC,landSeaMask,time_dsc,dsf,plot_year,maxVal,settings)
plt.title(str(plot_year) + '\nRCP8.5 noise', fontsize=16)            

plot_year = 2030
ax2 = fig.add_subplot(gs[0,1],projection=mapProj)
plots.plot_noise(ax2,featuresC,featuresF,featuresC,landSeaMask,time_dsc,dsf,plot_year,maxVal,settings)
plt.title(str(plot_year) + '\nRCP8.5 noise', fontsize=16)                    

plot_year = 2021
ax3 = fig.add_subplot(gs[1,0],projection=mapProj)
plots.plot_noise(ax3,featuresC,featuresF,featuresF,landSeaMask,time_dsc,dsf,plot_year,maxVal,settings)
plt.title(str(plot_year) + '\nGLENS-SAI noise', fontsize=16)                       

plot_year = 2030
ax4 = fig.add_subplot(gs[1,1],projection=mapProj)
plots.plot_noise(ax4,featuresC,featuresF,featuresF,landSeaMask,time_dsc,dsf,plot_year,maxVal,settings)
plt.title(str(plot_year) + '\nGLENS-SAI noise', fontsize=16)                       

plt.tight_layout()
plt.savefig('figures/' + filename + '_noise_panels.png', dpi=dpiFig, bbox_inches = 'tight')
plt.show()

In [ ]:
error('here')

# Mean change

In [ ]:
# from palettable.cmocean import Balance_15
cmap_div = palettable.cmocean.diverging.Balance_15.mpl_colormap
fastBool = True
titleSize = 16
gaussian_filter_sigma = .0

#-----------------------------------------------
maps_C = copy.deepcopy(featuresC)
maps_F = copy.deepcopy(featuresF)
if(settings["land_only"]==True):
    maps_C = copy.deepcopy(maps_C*landSeaMask.reshape(featuresC.shape[2],featuresC.shape[3]))
    maps_F = copy.deepcopy(maps_F*landSeaMask.reshape(featuresC.shape[2],featuresC.shape[3]))    
maps_diff = maps_F - maps_C

i_nonan = np.count_nonzero(~np.isnan(maps_C[0,0,:,:]))
mean_vals_C = np.nansum(np.nansum(maps_C,axis=3),axis=2)/i_nonan
maps_C_nomean = copy.deepcopy(maps_C - mean_vals_C[:,:,np.newaxis,np.newaxis])
mean_vals_F = np.nansum(np.nansum(maps_F,axis=3),axis=2)/i_nonan
maps_F_nomean = copy.deepcopy(maps_F - mean_vals_F[:,:,np.newaxis,np.newaxis])
maps_diff_nomean = maps_F_nomean - maps_C_nomean

for meantype in ('withMean',):
    
    if(settings["var"]!='T' and meantype=='noMean'):
        continue
    
    if(settings["var"]=='T'):
        maxVal = 310.
        minVal = 230.
        maxVal_change = 5.        
        cb_ticks = np.asarray(np.arange(minVal,maxVal+20,20),dtype='int')
        cb_ticks_both = np.asarray(np.arange(-maxVal_change,maxVal_change+5,5),dtype='int')
        units = '$^o$C'
        factor = 1.
        cmap = palettable.cmocean.sequential.Matter_15.mpl_colormap        
        plot_year = 2030
    elif(settings["var"]=='TX90p' or settings["var"]=='TX10p'):
        maxVal = 25   
        minVal = 0
        maxVal_change = maxVal
        minVal_change = minVal
        cb_ticks = np.arange(0,maxVal+5,5)
        cb_ticks_both = np.arange(-maxVal_change,maxVal_change+5,5)
        units = '\%'        
        factor = 100.
        cmap = palettable.cmocean.sequential.Matter_15.mpl_colormap        
        plot_year = 2030        
    elif(settings["var"]=='R95pTOT'):
        maxVal = 1000.
        minVal = 0        
        maxVal_change = maxVal
        minVal_change = minVal        
        cb_ticks = np.arange(0,maxVal+500,500)
        cb_ticks_both = np.arange(-maxVal_change,maxVal_change+500,500)
        units = 'mm'
        factor = 1.
        cmap = palettable.cmocean.sequential.Thermal_15_r.mpl_colormap
        plot_year = 2030        
    #-----------------------------------------------         
    fig = plt.figure(figsize=(12,7.75))        
    xplot = []
    #----------------------------------------------- 
    itime = np.where(time_dsc==plot_year)[0]
    if(meantype=='withMean'):
        xplot = maps_C[:,itime,:,:]
    elif(meantype=='noMean'):
        xplot = maps_C_nomean[:,itime,:,:]
    else:
        raise ValueError('no such meantype')
    xplot = np.squeeze(np.mean(xplot[(-1,),:,:,:],axis=0))
    print('map mean = ' + str(np.nanmean(xplot[:])))

    ax1 = plt.subplot(2,2,1,projection=mapProj)
    cb1, image = plots.drawOnGlobe(ax1, xplot*factor, dsf['lat'].values, dsf['lon'].values, cmap=cmap, vmin = minVal, vmax=maxVal, cbarBool=True, fastBool=fastBool, extent='max')
    cb1.set_label(units, fontsize=titleSize/2)
    cb1.set_ticks(cb_ticks)
    cb1.set_ticklabels(cb_ticks)            
    cb1.ax.tick_params(labelsize=titleSize/2) 
    plt.title('(a) RCP8.5\n' + str(plot_year) + ', Member \#1',fontsize=titleSize)                     

    #-----------------------------------------------
    itime = np.where(time_dsc==plot_year)[0]
    if(meantype=='withMean'):
        xplot = maps_F[:,itime,:,:]
    elif(meantype=='noMean'):
        xplot = maps_F_nomean[:,itime,:,:]
    else:
        raise ValueError('no such meantype')
    xplot = np.squeeze(np.mean(xplot[(-1,),:,:,:],axis=0))

    ax2 = plt.subplot(2,2,2,projection=mapProj)
    cb2, image = plots.drawOnGlobe(ax2, xplot*factor, dsf['lat'].values, dsf['lon'].values, cmap=cmap, vmin = minVal, vmax=maxVal, cbarBool=True, fastBool=fastBool, extent='max')
    cb2.set_label(units, fontsize=titleSize/2)
    cb2.set_ticks(cb_ticks)
    cb2.set_ticklabels(cb_ticks)            
    cb2.ax.tick_params(labelsize=titleSize/2) 
    plt.title('(b) GLENS-SAI\n' + str(plot_year) + ', Member \#1',fontsize=titleSize)                     

    #-----------------------------------------------
    itime = np.where(time_dsc==plot_year)[0]
    if(meantype=='withMean'):
        xplot = maps_diff[:,itime,:,:]
    elif(meantype=='noMean'):
        xplot = maps_diff_nomean[:,itime,:,:]
    else:
        raise ValueError('no such meantype')
    xplot = np.squeeze(np.mean(xplot[(-1),:,:,:],axis=0))

    ax3 = plt.subplot(2,2,3,projection=mapProj)
    cb3, image = plots.drawOnGlobe(ax3, xplot*factor, dsf['lat'].values, dsf['lon'].values, cmap=cmap_div, vmin = -maxVal_change, vmax=maxVal_change, cbarBool=True, fastBool=fastBool, extent='both')
    cb3.set_label(units, fontsize=titleSize/2)
    cb3.set_ticks(cb_ticks_both)
    cb3.set_ticklabels(cb_ticks_both)            
    cb3.ax.tick_params(labelsize=titleSize/2) 
    plt.title('(c) GLENS-SAI - RCP8.5\n' + str(plot_year) + ', Member \#1',fontsize=titleSize)                     
    
    #-----------------------------------------------
    itime = np.where(time_dsc==plot_year)[0]
    if(meantype=='withMean'):
        xplot = maps_diff[:,itime,:,:]
    elif(meantype=='noMean'):
        xplot = maps_diff_nomean[:,itime,:,:]
    else:
        raise ValueError('no such meantype')
    xplot = np.squeeze(np.mean(xplot[:,:,:,:],axis=0))
    print('map mean = ' + str(np.nanmean(xplot[:])))

    ax4 = plt.subplot(2,2,4,projection=mapProj)
    cb4, image = plots.drawOnGlobe(ax4, xplot*factor, dsf['lat'].values, dsf['lon'].values, cmap=cmap_div, vmin = -maxVal_change, vmax=maxVal_change, cbarBool=True, fastBool=fastBool, extent='both')
    cb4.set_label(units, fontsize=titleSize/2)
    cb4.set_ticks(cb_ticks_both)
    cb4.set_ticklabels(cb_ticks_both)            
    cb4.ax.tick_params(labelsize=titleSize/2) 
    plt.title('(d) GLENS-SAI - RCP8.5\n' + str(plot_year) + ', Ensemble Mean',fontsize=titleSize)                     

    plt.tight_layout()
    plt.savefig('figures/' + filename + '_maps_comparisons2030_' + meantype + '.png', dpi=dpiFig, bbox_inches = 'tight')

    plt.show()


# Temperature time series

In [ ]:
FS = 16
sai_colors = palettable.colorbrewer.sequential.PuBu_5
rcp_colors = palettable.colorbrewer.sequential.OrRd_5


if settings["var"] == 'T':
    lat = dsf['lat']
    lon = dsf['lon']
    weights = np.cos(np.deg2rad(lat))


    ################################  
    ################################           
    fig = plt.figure(figsize=(6,4.))
    ax = plt.subplot(111)
    ################################  
    ################################  
    plots.adjust_spines(ax, ['left', 'bottom'])
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.spines['left'].set_color('dimgrey')
    ax.spines['bottom'].set_color('dimgrey')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ################################  
    ################################  
    ax.tick_params('both',length=4,width=2,which='major',color='dimgrey')
    ax.yaxis.grid(zorder=1,color='dimgrey',alpha=0.35)
    ################################  
    ################################


    # plot Control
    itime = np.where(time_dsc==2021)[0]
    data_zm = np.nanmean(featuresC,axis=-1)
    data_mean_C = np.average(data_zm,axis=2,weights=weights)
    anom_val = np.nanmean(data_mean_C[:,itime],axis=0)
    data_mean_anom_C = data_mean_C - anom_val
    plt.plot(-100,100,linewidth=1,color=rcp_colors.hex_colors[-1],label='RCP8.5')
    plt.plot(time_dsc,np.swapaxes(data_mean_anom_C,1,0),linewidth=1,color=rcp_colors.hex_colors[-1])


    # plot Feedback
    itime = np.where(time_dsf==2021)[0]
    data_zm = np.nanmean(featuresF,axis=-1)
    data_mean_F = np.average(data_zm,axis=2,weights=weights)
    anom_val = np.nanmean(data_mean_F[:,itime],axis=0)
    data_mean_anom_F = data_mean_F - anom_val
    plt.plot(-100,100,linewidth=1,color=sai_colors.hex_colors[-1],label='GLENS-SAI')
    plt.plot(time_dsf,np.swapaxes(data_mean_anom_F,1,0),linewidth=1,color=sai_colors.hex_colors[-1])

    # extra
    # plt.xlabel('year', fontsize=FS)
    plt.ylabel('$^o$C anomaly from 2021', fontsize=FS)
    array_of_ylabels = (-1,0,1,2,3,4,5)
    array_of_xlabels = np.arange(2020,2100,10)
    plt.xticks(array_of_xlabels,array_of_xlabels,fontsize=FS/2)
    plt.yticks(array_of_ylabels,array_of_ylabels,fontsize=FS/2)
    plt.xlim(2019, 2098)
    plt.ylim(-1, 5.)

    plt.title('Global Mean\n1000 hPa temperature',fontsize=FS)
    plt.legend(fontsize=FS/2,
               loc="upper center",
               bbox_to_anchor=(.5, 1.01),
               ncol=2,
               frameon=False)

    plt.tight_layout()
    plt.savefig('figures/' + filename + '_global_mean.png', dpi=dpiFig, bbox_inches = 'tight')
    plt.show()

